In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys, os
sys.path.append(os.path.abspath("../.."))
from configs import GOOGLE_APPLICATION_CREDENTIALS,GCS_BUCKET_NAME,GCS_PROJECT_ID
from google.cloud import bigquery
from src.utils.io_utils import upload_to_bigquery
from clean_utils import *

In [2]:
client = bigquery.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
table_id = f"{GCS_PROJECT_ID}.{GCS_BUCKET_NAME}.data_train_model"
table_id_done = f"{GCS_PROJECT_ID}.{GCS_BUCKET_NAME}.data_done"

In [3]:
query = """SELECT *
FROM `khangtestdbt.xecupredict.data_cleaned` """
data_cleaned = client.query(query).to_dataframe()
data_cleaned.head(1)

,km,origin,body,fuel,name,price,brand,age
0,279,nhập khẩu,sedan,Xăng,Volvo S90 Ultra B6 AWD 2025 2 Tỷ 99 Triệu,2099000000,volvo,0


In [4]:
df = data_cleaned.copy()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16159 entries, 0 to 16158
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   km      16159 non-null  Int64 
 1   origin  16143 non-null  object
 2   body    14515 non-null  object
 3   fuel    14793 non-null  object
 4   name    16159 non-null  object
 5   price   16159 non-null  Int64 
 6   brand   16159 non-null  object
 7   age     16159 non-null  Int64 
dtypes: Int64(3), object(5)
memory usage: 1.0+ MB


In [6]:
df.duplicated().sum()

np.int64(655)

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df.isna().sum()

km           0
origin      16
body      1636
fuel      1337
name         0
price        0
brand        0
age          0
dtype: int64

1.brand

In [9]:
df["brand"].dtype

dtype('O')

In [10]:
df["brand"].unique()

array(['volvo', 'vinfast', 'volkswagen', 'landrover', 'kia',
       'lamborghini', 'lexus', 'ford', 'honda', 'gaz', 'hongqi', 'hummer',
       'hyundai', 'bmw', 'chevrolet', 'citroen', 'daewoo', 'daihatsu',
       'dodge', 'ferrari', 'audi', 'baic', 'bentley', 'byd', 'cadillac',
       'acura', 'aston', 'infiniti', 'isuzu', 'jaguar', 'jeep', 'kenbo',
       'mazda', 'lynk', 'maserati', 'mercedes', 'mclaren', 'toyota',
       'suzuki', 'sym', 'thaco', 'mitsubishi', 'porsche', 'peugeot',
       'nissan', 'omoda', 'ram', 'renault', 'rolls', 'samsung', 'skoda',
       'smart', 'ssangyong', 'subaru', 'mg', 'mini', 'mercedes benz',
       'lynk&co', 'alfa romeo', 'gac', 'hãng khác', 'wuling', 'changan',
       'zotye', 'reult', 'luxgen', 'rolls royce', 'geely', 'genesis',
       'dongfeng', 'chrysler', 'mekong', 'fiat', 'asia', 'lada', 'srm',
       'land', 'dongben'], dtype=object)

In [11]:
brand_alias = {
    "mercedes": "mercedes-benz",
    "mercedes benz": "mercedes-benz",

    "rolls": "rolls-royce",
    "rolls royce": "rolls-royce",

    "land": "land rover",
    "landrover": "land rover",

    "lynk": "lynk & co",
    "lynk&co": "lynk & co",
}

In [12]:
df["brand"] = df["brand"].replace(brand_alias)

top_brands = df["brand"].value_counts().nlargest(25).index
df["brand"] = df["brand"].apply(lambda x: x if x in top_brands else "other")


In [13]:
df["brand"].value_counts()

brand
toyota           2831
ford             1713
mercedes-benz    1646
kia              1472
hyundai          1380
mitsubishi        934
mazda             778
vinfast           769
honda             659
lexus             555
bmw               460
chevrolet         314
suzuki            228
other             226
porsche           222
peugeot           213
land rover        185
nissan            178
mg                176
audi              171
volvo             116
volkswagen         98
daewoo             67
isuzu              44
mini               43
jaguar             26
Name: count, dtype: int64

2.origin

In [14]:
df["origin"].unique()

array(['nhập khẩu', 'trong nước', None], dtype=object)

In [15]:
df["origin"].value_counts()

origin
nhập khẩu     8340
trong nước    7148
Name: count, dtype: int64

In [16]:
origin_mode_per_brand = df.groupby('brand')['origin'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
origin_mode_per_brand


brand
audi              nhập khẩu
bmw               nhập khẩu
chevrolet         nhập khẩu
daewoo            nhập khẩu
ford              nhập khẩu
honda             nhập khẩu
hyundai          trong nước
isuzu             nhập khẩu
jaguar            nhập khẩu
kia              trong nước
land rover        nhập khẩu
lexus             nhập khẩu
mazda            trong nước
mercedes-benz    trong nước
mg                nhập khẩu
mini              nhập khẩu
mitsubishi        nhập khẩu
nissan            nhập khẩu
other             nhập khẩu
peugeot          trong nước
porsche           nhập khẩu
suzuki            nhập khẩu
toyota            nhập khẩu
vinfast          trong nước
volkswagen        nhập khẩu
volvo             nhập khẩu
Name: origin, dtype: object

In [17]:
df['origin'] = df.apply(
    lambda row: origin_mode_per_brand[row['brand']] if pd.isna(row['origin']) else row['origin'],
    axis=1
)


In [18]:
df['origin'].isna().sum()

np.int64(0)

3.body

In [19]:
df["body"].unique()

array(['sedan', 'suv', 'van/minivan', 'coupe', 'hatchback', 'crossover',
       'convertible/cabriolet', 'bán tải / pickup', 'truck',
       'suv / cross over', 'pick-up (bán tải)', None, 'minivan (mpv)',
       'kiểu dáng khác', 'van', 'coupe (2 cửa)', 'mui trần', 'mpv',
       'special purpose', 'bán tải', 'convertible', 'sport car', 'xe tải',
       'minibus'], dtype=object)

In [20]:
df['body'] = df['body'].replace({
    'xe tải': 'truck',
    'truck': 'truck',
    'bán tải': 'pickup',
    'bán tải / pickup': 'pickup',
    'pick-up (bán tải)': 'pickup',
    'van/minivan': 'minivan',
    'minivan (mpv)': 'minivan',
    'mpv': 'minivan',
    'suv / cross over': 'suv',
    'crossover': 'suv',
    'convertible/cabriolet': 'convertible',
    'coupe (2 cửa)': 'coupe'
})


In [21]:
body_mode_per_brand = df.groupby('brand')['body'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
body_mode_per_brand

brand
audi                   suv
bmw                  sedan
chevrolet              suv
daewoo               sedan
ford                   suv
honda                sedan
hyundai                suv
isuzu                  suv
jaguar               sedan
kia                    suv
land rover             suv
lexus                  suv
mazda                sedan
mercedes-benz        sedan
mg                     suv
mini             hatchback
mitsubishi             suv
nissan                 suv
other                  suv
peugeot                suv
porsche                suv
suzuki           hatchback
toyota                 suv
vinfast                suv
volkswagen             suv
volvo                  suv
Name: body, dtype: object

In [22]:
df['body'] = df.apply(
    lambda row: body_mode_per_brand[row['brand']] if pd.isna(row['body']) else row['body'],
    axis=1
)

In [23]:
df['body'].isna().sum()

np.int64(0)

4.fuel

In [24]:
df["fuel"].unique()

array(['Xăng', 'Hybrid', 'Điện', 'Dầu', None], dtype=object)

In [25]:
df['fuel'] = df['fuel'].str.lower()

In [26]:
fuel_mode_per_brand_body = df.groupby(["brand","body"])['fuel'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
def fill_fuel(row):
    if pd.isna(row['fuel']):
        return fuel_mode_per_brand_body.get((row['brand'], row['body']), None)
    else:
        return row['fuel']

df['fuel'] = df.apply(fill_fuel, axis=1)

In [27]:
df['fuel'].isna().sum()

np.int64(8)

In [28]:
fuel_mode_per_brand = df.groupby('brand')['fuel'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
df['fuel'] = df.apply(
    lambda row: fuel_mode_per_brand[row['brand']] if pd.isna(row['fuel']) else row['fuel'],
    axis=1
)

df['fuel'].isna().sum()

np.int64(0)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15504 entries, 0 to 16158
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   km      15504 non-null  Int64 
 1   origin  15504 non-null  object
 2   body    15504 non-null  object
 3   fuel    15504 non-null  object
 4   name    15504 non-null  object
 5   price   15504 non-null  Int64 
 6   brand   15504 non-null  object
 7   age     15504 non-null  Int64 
dtypes: Int64(3), object(5)
memory usage: 1.1+ MB


5.outlier

In [30]:
df["price"] = df["price"].astype(float)
df["km"] = df["km"].astype(float)
df["age"] = df["age"].astype(float)

In [31]:
def cap_outliers_group(df, column, group_cols):
    def cap_group(x):
        Q1 = x[column].quantile(0.25)
        Q3 = x[column].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        return x[column].clip(lower, upper)
    
    df[column] = df.groupby(group_cols, group_keys=False).apply(cap_group)
    return df

In [32]:
df = cap_outliers_group(df, "km", ['age'])
df = cap_outliers_group(df, "price", ['brand','age'])

In [33]:
df.head()

,km,origin,body,fuel,name,price,brand,age
0,279.0,nhập khẩu,sedan,xăng,Volvo S90 Ultra B6 AWD 2025 2 Tỷ 99 Triệu,2.099000e+09,volvo,0.0
1,4000.0,nhập khẩu,sedan,xăng,Volvo S90 Ultra B6 AWD 2025 2 Tỷ 39 Triệu,2.039000e+09,volvo,0.0
2,120000.0,nhập khẩu,suv,xăng,Volvo XC40 T5 AWD R Design 2019 1 Tỷ 55 Triệu,1.055000e+09,volvo,6.0
3,32000.0,nhập khẩu,suv,xăng,Volvo XC40 Ultimate B5 AWD 2022 1 Tỷ 489 Triệu,1.489000e+09,volvo,3.0
4,17900.0,nhập khẩu,suv,xăng,Volvo XC40 Ultimate B5 AWD 2023 1 Tỷ 559 Triệu,1.559000e+09,volvo,2.0


In [34]:
df.drop(columns=["name"], inplace=True)

In [35]:
X = df.drop(columns=["price"])
X.head()

,km,origin,body,fuel,brand,age
0,279.0,nhập khẩu,sedan,xăng,volvo,0.0
1,4000.0,nhập khẩu,sedan,xăng,volvo,0.0
2,120000.0,nhập khẩu,suv,xăng,volvo,6.0
3,32000.0,nhập khẩu,suv,xăng,volvo,3.0
4,17900.0,nhập khẩu,suv,xăng,volvo,2.0


In [36]:
X[X.duplicated(keep=False)]


,km,origin,body,fuel,brand,age
5,17000.0,nhập khẩu,suv,xăng,volvo,2.0
6,80000.0,nhập khẩu,suv,xăng,volvo,7.0
9,120000.0,nhập khẩu,suv,xăng,volvo,7.0
19,60000.0,nhập khẩu,suv,hybrid,volvo,3.0
21,90000.0,nhập khẩu,suv,hybrid,volvo,3.0
...,...,...,...,...,...,...
16152,25000.0,trong nước,minivan,dầu,kia,1.0
16154,23000.0,trong nước,sedan,xăng,kia,1.0
16155,3000.0,trong nước,suv,xăng,hyundai,0.0
16156,12000.0,trong nước,suv,xăng,hyundai,0.0


In [37]:
df_unique = df.groupby(list(X.columns), as_index=False)["price"].mean()
df_unique

,km,origin,body,fuel,brand,age,price
0,105.0,trong nước,minivan,xăng,toyota,14.0,205000000.0
1,107.0,trong nước,minivan,xăng,toyota,15.0,235000000.0
2,110.0,nhập khẩu,hatchback,xăng,hyundai,11.0,232000000.0
3,110.0,nhập khẩu,sedan,xăng,toyota,18.0,295000000.0
4,112.0,nhập khẩu,mui trần,xăng,mini,20.0,365000000.0
...,...,...,...,...,...,...,...
11300,350000.0,nhập khẩu,suv,xăng,mitsubishi,35.0,450000000.0
11301,358950.0,nhập khẩu,sedan,xăng,mitsubishi,25.0,65000000.0
11302,412027.5,nhập khẩu,suv,xăng,toyota,24.0,97500000.0
11303,451000.0,nhập khẩu,sedan,xăng,toyota,27.0,92500000.0


In [38]:
cols_show = ["brand", "body", "fuel", "origin"]

df_unique[cols_show] = df_unique[cols_show].apply(lambda c: c.str.title())


In [39]:
df_unique.sample(10)

,km,origin,body,fuel,brand,age,price
5847,65000.0,Nhập Khẩu,Sedan,Xăng,Bmw,11.0,6.890000e+08
10500,150000.0,Nhập Khẩu,Hatchback,Xăng,Toyota,16.0,2.250000e+08
3625,40000.0,Nhập Khẩu,Suv,Xăng,Toyota,6.0,7.750000e+08
6171,68068.0,Trong Nước,Suv,Xăng,Mitsubishi,6.0,5.800000e+08
1800,19000.0,Trong Nước,Suv,Dầu,Kia,1.0,1.128000e+09
5733,63000.0,Nhập Khẩu,Suv,Xăng,Porsche,3.0,2.899000e+09
2991,32151.0,Trong Nước,Sedan,Xăng,Chevrolet,9.0,2.550000e+08
2877,31000.0,Nhập Khẩu,Suv,Dầu,Kia,1.0,1.365000e+09
4581,50300.0,Nhập Khẩu,Sedan,Xăng,Mazda,8.0,4.000000e+08
1980,20000.0,Trong Nước,Suv,Điện,Vinfast,2.0,8.763333e+08


In [40]:
df_unique.duplicated().sum()

np.int64(0)

In [41]:
import json


unique_values = {
    'origin': df_unique['origin'].value_counts().index.tolist(),
    'fuel':   df_unique['fuel'].value_counts().index.tolist(),
    'body':   df_unique['body'].value_counts().index.tolist(),
    'brand':  df_unique['brand'].value_counts().index.tolist(),
}

with open('../../model/unique_values.json', 'w', encoding='utf-8') as f:
    json.dump(unique_values, f, ensure_ascii=False, indent=2)

In [42]:
upload_to_bigquery(df_unique, table_id=table_id_done, if_exists="replace" )

✅ Uploaded 11305 rows to khangtestdbt.xecupredict.data_done


# end